# Demo related to the paper "RAGent: Retrieval-based Access Control Policy Generation"


> Estimated time: 30 minutes

In this demo, we utilize RAGent to identify NLACPs from real-world high-level requirements of HotCRP.com privacy policies and translate them into access control policies.

---

<details>
  <summary>HotCRP Privacy Policy</summary>

  This page explains how <u>[HotCRP.com](https://hotcrp.com/)</u> uses the data you provide and how you can control that use. The open-source HotCRP software also runs on sites we don’t manage; this policy applies only to sites with domains ending in “.hotcrp.com”.  
(Updated 2020-08-24)  

# What we store and why

The HotCRP.com service handles the submission and review of *artifacts*, such as scientific and engineering publications. In most cases, HotCRP.com does not own the data you provide. Rather, we broker that data on behalf of scholarly societies, such as ACM and USENIX; on behalf of conference review boards; and on behalf of other site managers.
- We store **Submission artifacts**, including submissions (such as PDFs), metadata (such as titles and author lists), and supplementary information (such as topics and other uploaded files).
- We store **Review artifacts**, including reviews, discussions, and associated uploaded files.
- We store **Configuration settings** used by site managers to configure a site.
- We store **Profile data**, such as user names, affiliations, email addresses, topic interests, and review preferences.
- We store **Demographic data**, such as user gender identity and approximate age.
- We store **Browsing data**, such as logs of which sites a computer has accessed.

# Controlling your information

**Submission and review artifacts**: When you submit artifacts to a HotCRP.com site, you give that site’s managers (e.g., the program chairs) permission to store and view those artifacts indefinitely. You give the site managers permission to distribute the artifacts at their discretion, for review or other purposes. Finally, you give HotCRP.com permission to process the artifacts and to store them indefinitely. Site managers control who can access artifacts, and HotCRP.com doesn’t share artifacts except as site managers allow. However, we may publicize aggregate information that cannot be traced to any site or user, such as total submission counts across all sites. If an artifact was submitted in error, you can request its permanent deletion. Such requests should be directed to the relevant site managers (e.g., program chairs).

**Configuration settings** are stored by HotCRP.com indefinitely. Site managers may request the deletion of a HotCRP.com site, if allowed by their site agreements with HotCRP.com. This will delete all associated site data, including submission and review artifacts.

**Profile data** is stored in several ways.
- Every HotCRP.com user has a **global profile**, which includes name, email, and affiliation, and other information.
- Upon submitting an artifact to a HotCRP.com site, a user gains a **site profile** for that site. This generally contains the same information as the global profile, but it can differ. (For example, changes to a global user affiliation only affect future site profiles.)

Users can update their profiles at any time. A site manager can also create a profile for a user, for example by inviting them to join a conference program committee. Contact us if you want your global profile deleted and your site profiles disabled.  

**Demographic data** is stored in user global profiles, and can only be modified by users themselves (never by site managers). Users control what demographic data is stored and how demographic data is shared using the Profile \> Demographics tab on any HotCRP.com site. Information shared only with scholarly societies is provided directly to those societies and cannot be accessed by site managers. Information that is explicitly shared with site managers may also be analyzed by HotCRP.com, but will be published only in aggregate, such as in terms of percentages of active HotCRP.com users.  

**Browsing data** is stored for up to a month and per-user browsing data is never shared. We may store and share aggregate information such as total page loads. Misbehavior, such as denial-of-service attacks and attempts to crack a user’s password, may be publicized and may be preserved indefinitely.


</details>

---

Finally, before we begin, since LLaMa 3 8B is a gated model, you have visit [huggingface](https://huggingface.co/meta-llama/Meta-Llama-3-8B) and get the access to the base model.


## Install and import necessary libraries

In [ ]:
!pip install -U spacy-experimental
!pip install https://github.com/explosion/spacy-experimental/releases/download/v0.6.1/en_coreference_web_trf-3.4.0a2-py3-none-any.whl
!pip install transformers==4.30.2
!pip install nltk
!pip install gdown==5.1.0

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

import spacy
import json
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# Change the following path appropriately
PATH_DEMO_DATA = 'privacy_hotcrp.md'

## Pre-processing the input privacy policy document

This involves seperating the privacy policy document into paragraphs, resolving co-references in each paragraph, and segmenting each paragraph into individual sentences.

In [ ]:
nlp = spacy.load('en_coreference_web_trf')

def resolve_references(sent):
    """Function for resolving references with the coref ouput
    doc (Doc): The Doc object processed by the coref pipeline
    RETURNS (str): The Doc string with resolved references
    """

    doc = nlp(sent)
    token_mention_mapper = {}
    output_string = ""
    clusters = [
        val for key, val in doc.spans.items() if key.startswith("coref_cluster")
    ]

    for cluster in clusters:
        first_mention = cluster[0]
        for mention_span in list(cluster)[1:]:
            token_mention_mapper[mention_span[0].idx] = first_mention.text + mention_span[0].whitespace_

            for token in mention_span[1:]:
                token_mention_mapper[token.idx] = ""

    for token in doc:
        if token.idx in token_mention_mapper:
            output_string += token_mention_mapper[token.idx]
        else:
            output_string += token.text + token.whitespace_

    return output_string

In [ ]:
with open(PATH_DEMO_DATA, 'r+') as f:
    content = f.read().replace('*','').replace('#','').replace('-','').replace('\xa0',' ')

In [ ]:
paragraphs = content.split('\n\n')
coref_resolved = [resolve_references(k) for k in paragraphs]

In [ ]:
preprocessed_lines = []
for p in coref_resolved:
    preprocessed_lines.extend(p.split('\n'))
sents = []
for p in preprocessed_lines:
    sents.extend(sent_tokenize(p))

with open('high_level_requirements.json', 'w') as f:
  json.dump(sents, f)

## Running RAGent

After identifying individual sentences, then we can run RAGent on these sentences. To this end, we first have to install and import extra libraries.

In [ ]:
!pip install -U transformers
!pip install -U peft
!pip install bitsandbytes
!pip install einops
!pip install captum
!pip install langchain

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "3"

import torch
from peft import PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    BertForSequenceClassification,
    BertTokenizerFast
)
from captum.attr import LayerIntegratedGradients, visualization
import json
import pandas as pd
from tqdm import tqdm

device = 'cuda:0'

CACHE = '/data/sjay950/huggingface/'

### Loading the NLACP identification model

In [ ]:
ID_MODEL_NAME = '../checkpoints/identification/overall/checkpoint/'
NUM_CLASSES = 2

id_model = BertForSequenceClassification.from_pretrained(ID_MODEL_NAME, num_labels=NUM_CLASSES).to(device)
id_tokenizer = BertTokenizerFast.from_pretrained(ID_MODEL_NAME)

id_model.eval()

### Loading the access control policy generation model

In [ ]:
GEN_MODEL_NAME = 'meta-llama/Meta-Llama-3-8B-Instruct'
GEN_MODEL_CKPT = '../checkpoints/generation/overall/checkpoint/'

model_kwargs = dict(
    trust_remote_code=True,
    attn_implementation="flash_attention_2",  # loading the model with flash-attenstion support
    torch_dtype=torch.bfloat16,
    cache_dir = CACHE
)
base_model = AutoModelForCausalLM.from_pretrained(GEN_MODEL_NAME, **model_kwargs)

gen_model = PeftModel.from_pretrained(base_model, GEN_MODEL_CKPT).to('cuda:0')

gen_tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL_NAME, use_fast=True, cache_dir=CACHE)
gen_tokenizer.pad_token = gen_tokenizer.eos_token
gen_tokenizer.padding_side = 'left'

### Loading the access control policy verification model

In [ ]:
VER_MODEL = "facebook/bart-large"
VER_MODEL_CKPT = "../checkpoints/verification/checkpoint/"

ver_tokenizer = AutoTokenizer.from_pretrained(VER_MODEL, cache_dir = CACHE)

ver_model = AutoModelForSequenceClassification.from_pretrained(VER_MODEL_CKPT).to(
    "cuda:0"
)
ver_model = ver_model.eval()

### Supporting function definition for explaining the verification result through feature attribution


In [ ]:
def fwd_function(input_ids, attention_mask):

    inp = {'input_ids': input_ids.to(device), 'attention_mask': attention_mask.to(device)}

    # with torch.no_grad():
    pred = ver_model(**inp).logits

    return pred


def construct_input_ref(s,l, ref_token_id, sep_token_id, bos_token_id):
    toks = ver_tokenizer.encode(s)
    tokl = ver_tokenizer.encode(l)

    input_ids = ver_tokenizer.encode(s,l)

    # construct reference token ids
    ref_input_ids = [bos_token_id] + [ref_token_id] * (len(toks)-2) + [sep_token_id] + [sep_token_id] + (len(tokl)-2)*[ref_token_id] + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device)

def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)


def clean(tokens):
    l = []
    for t in tokens:
        if t.startswith('Ġ'):
            l.append(t[1:])
        else:
            l.append(t)
    return l

The below cell defines the layer integrated gradients instance using the verifier's forward methos and its embeddings.

In [ ]:
lig = LayerIntegratedGradients(fwd_function, ver_model.base_model.encoder.embed_tokens)

The below cell defines functions need to store and visualize feature attributions by the Captum library.

In [ ]:
from utils import ID2AUGS

vis_data_records_ig = []

def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            ID2AUGS[pred_ind],
                            ID2AUGS[label],
                            ID2AUGS[pred_ind],
                            attributions.sum(),
                            text,
                            delta))

def interpret_sentence(sentence, policy, tokenizer, pred_ind, pred, label):

    input_ids, input_ref = construct_input_ref(sentence, policy, tokenizer.pad_token_id, tokenizer.sep_token_id, tokenizer.bos_token_id)

    indices = input_ids[0].detach().tolist()
    text = clean(tokenizer.convert_ids_to_tokens(indices))

    attention_mask = construct_attention_mask(input_ids)

    attributions_ig, delta = lig.attribute(inputs=input_ids,
                                           baselines=input_ref,
                                           additional_forward_args=(attention_mask),
                                           n_steps=500,
                                           target=pred_ind,
                                           internal_batch_size = 4,
                                           return_convergence_delta=True)

    print('pred: ', ID2AUGS[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))
    add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)

### Executing RAGentV

In [ ]:
with open('high_level_requirements.json', 'r') as f:
    sents = json.load(f)

In [ ]:
!mkdir -p generations

In [ ]:
from utils import create_vectorstores

stores = create_vectorstores('entities.json', True)

In [ ]:
from utils import generate_policy, is_nlacp, convert_to_sent

inputs, outputs = [], []

id = 0
for s in tqdm(sents):
    inputs.append(s)
    nlacp = is_nlacp(s, id_model, id_tokenizer)
    if nlacp:
        output, pred_ind, mprob = generate_policy(id, s, gen_model, gen_tokenizer, ver_model, ver_tokenizer, with_ents=True, store=stores)
        if pred_ind != 11:
            pp, _ = convert_to_sent(str(output))
            interpret_sentence(s, pp, ver_tokenizer, pred_ind, mprob, pred_ind)

    else:
        output = 'Not an ACP'
    
    outputs.append(output)
    id+=1

df = pd.DataFrame({
    'inputs': inputs,
    'outputs': outputs
})

df.to_csv('results.csv')

In [ ]:
_ = visualization.visualize_text(vis_data_records_ig)